In [ ]:
!pip install pylibiio pyadi-iio

In [ ]:
import adi
import numpy as np

fi = int(1e5)
fc = int(2.5e9)
fs = int(16e6)
rx_mode = "fast_attack"
rx_n = int(2**9)

pluto_ip = "192.168.1.17"
sdr_rxtx = adi.ad9361(uri="ip:%s" % pluto_ip)

rx_gain = 0
rx_lo = fc

tx_lo = fc

In [ ]:
sdr_rxtx.tx_hardwaregain_chan0 = -80

In [ ]:
sdr_rxtx.rx_enabled_channels = [0, 1]
sdr_rxtx.sample_rate = fs
sdr_rxtx.rx_rf_bandwidth = int(3 * fi)

sdr_rxtx.rx_lo = int(rx_lo)
sdr_rxtx.rx_hardwaregain_chan0 = int(rx_gain)
sdr_rxtx.rx_hardwaregain_chan1 = int(rx_gain)
sdr_rxtx.gain_control_mode = rx_mode
sdr_rxtx.rx_buffer_size = int(rx_n)
sdr_rxtx._rxadc.set_kernel_buffers_count(2)

In [ ]:
from math import gcd

sdr_rxtx.tx_rf_bandwidth = int(3 * fi)
sdr_rxtx.tx_lo = int(tx_lo)
sdr_rxtx.tx_enabled_channels = [0]
sdr_rxtx.tx_hardwaregain_chan0 = int(-50)  # tx_gain) #tx_gain)
sdr_rxtx.tx_hardwaregain_chan1 = int(-80)
tx_n = int(fs / gcd(fs, fi))
while tx_n < 1024 * 16:
    tx_n *= 2
# since its a cyclic buffer its important to end on a full phase
t = (
    np.arange(0, tx_n) / fs
)  # time at each point assuming we are sending samples at (1/fs)s
iq0 = np.exp(1j * 2 * np.pi * fi * t) * (2**14)
# try to reset the tx
sdr_rxtx.tx_destroy_buffer()
sdr_rxtx.tx_cyclic_buffer = True  # this keeps repeating!

In [ ]:
sdr_rxtx.rx()

In [ ]:
sdr_rxtx = None